In [ ]:
# import all dependencies
import requests
import pymongo
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [ ]:
# Initialize PyMongo to work with MongoBDs
conn = 'mongodb//localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

## NASA Mars News

In [ ]:
url = 'http://redplanetscience.com/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
print(soup.prettify())

In [ ]:
# scrape for news title and paragraph text
news_title = soup.find_all('div', class_ = 'content_title')[0].text
print(news_title)

In [ ]:
news_p = soup.find_all('div', class_ = 'article_teaser_body')[0].text
print(news_p)

## JPL Mars Space Images - Featured Image

In [ ]:
url = 'https://spaceimages-mars.com'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
print(soup.prettify())

In [ ]:
# scrape for featured image url
featured_image_tag = soup.find('img', class_ = 'headerimage fade-in')
featured_image_tag = featured_image_tag.attrs['src']
featured_image_tag

In [ ]:
featured_image_url = f'{url}/{featured_image_tag}'
featured_image_url

## Mars Facts

In [ ]:
url = 'https://galaxyfacts-mars.com'
facts = pd.read_html(url)
facts

In [ ]:
fact_table_df = facts[1]
fact_table_df = fact_table_df.rename(columns={0:"Profile",1:"Value"},errors="raise")
fact_table_df.set_index("Profile",inplace=True)
fact_table_df

In [ ]:
# convert pandas df to html string
html_fact_table = fact_table_df.to_html()
html_fact_table.replace('\n','')
print(html_fact_table)

## Mars Hemispheres

In [ ]:
# Scrape Mars hemisphere title and image
usgs_url='https://astrogeology.usgs.gov'
url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [ ]:
# Extract hemispheres item elements
mars_hems=soup.find('div',class_='collapsible results')
mars_item=mars_hems.find_all('div',class_='item')
hemisphere_image_urls=[]

In [ ]:
# Loop through each hemisphere item
for item in mars_item:
    # Error handling
    try:
        # Extract title
        hem=item.find('div',class_='description')
        title=hem.h3.text
        
        # Extract image url
        hem_url=hem.a['href']
        browser.visit(usgs_url+hem_url)
        html=browser.html
        soup=bs(html,'html.parser')
        image_src=soup.find('li').a['href']
        
        if (title and image_src):
            # Print results
            print('-'*50)
            print(title)
            print(image_src)
            
        # Create dictionary for title and url
        hem_dict={
            'title':title,
            'image_url':image_src
        }
        hemisphere_image_urls.append(hem_dict)
        
    except Exception as e:
        print(e)

In [ ]:
hem_dict

In [ ]:
# Create dictionary for all info scraped from sources above
mars_dict={
    "news_title":news_title,
    "news_p":news_p,
    "featured_image_url":featured_image_url,
    "fact_table":fact_table_df,
    "hemisphere_images":hem_dict
}
mars_dict